[View in Colaboratory](https://colab.research.google.com/github/aminzabardast/Tensorflow-Tutorials/blob/dev/4_Visualization_Using_Tensorboard.ipynb)

# Visualization Using Tensorboard

Visualization of structure and progress is an important part of designing an Artificial Neural Network (ANN). The aim of this notebook is to initiate the tensorflow user with visualization capabilities of Tensorflow.

The visualization tool included in Tensorflow package is named Tensorboard. Tensorflow documentation introduces Tensorboard as:

> The computations you'll use TensorFlow for - like training a massive deep neural network - can be complex and confusing. To make it easier to understand, debug, and optimize TensorFlow programs, we've included a suite of visualization tools called TensorBoard. You can use TensorBoard to visualize your TensorFlow graph, plot quantitative metrics about the execution of your graph, and show additional data like images that pass through it.

## Learning by example:

Importing necessary libraries.

In [1]:
import tensorflow as tf
import numpy as np

Generating Example Data. This example is the same as previous notebook. The only change is making the ANN generate necessary file for Tensorboard's visualization.

In [2]:
np.random.seed(seed=23)
xs = np.linspace(-1, 1, 300)[:, np.newaxis]
noise = np.random.normal(0, .1, size=xs.shape)
ys = np.square(xs) - 0.5 + noise

If we simply generate the files after initiating the session, the structure of the graph (ANN structure) would be as follow.

![Unstructured Network](https://raw.githubusercontent.com/aminzabardast/Tensorflow-Tutorials/dev/figures/4_unstructured_graph.png)

This is not informative and does not look like a neural network. To make the generated graph look better we have to modify the code.

### Changing `add_layer` function

We can group different parts of a graph together by using `tf.name_scope`.

For example, a whole layer can be encapsulated in a group and inside, weights and biases can be encapsulated in other seperate groups.
Below is the new code for adding layers.

In [3]:
def add_layer(input, in_size, out_size, name, activation_function=None, seed=None):
    with tf.name_scope('Layer_%s' % name):
        with tf.name_scope('Weights'):
            weights = tf.Variable(tf.random_normal([in_size, out_size], seed=seed))
        with tf.name_scope('Biases'):
            biases = tf.Variable(tf.zeros([1, out_size]) + 0.1)
        wx_plus_b = tf.matmul(input, weights) + biases
        output = activation_function(wx_plus_b) if activation_function else wx_plus_b
        return output

### Encapsulating Placeholders

Input vectors can be encapsulated as well.

In [4]:
with tf.name_scope('Input'):
    xp = tf.placeholder(tf.float32, shape=[None, 1])
with tf.name_scope('Truth'):
    yp = tf.placeholder(tf.float32, shape=[None, 1])

### Creating the Structures

In [5]:
l1 = add_layer(xp, 1, 3, '1', tf.nn.relu, seed=2)
l2 = add_layer(l1, 3, 10, '2', tf.nn.relu, seed=2)
l3 = add_layer(l2, 10, 3, '3', tf.nn.relu, seed=2)
prediction = add_layer(l3, 3, 1, 'Output', seed=2)

### Encapsulating and plotting Loss

Alongside encapsulating the loss for the graph, it is possible to define a scalar summerizer to keep track of value of loss. The result of this would become a loss vs epoch graph.

In [6]:
with tf.name_scope('Loss'):
    loss = tf.reduce_mean(tf.reduce_sum(tf.square(yp-prediction), reduction_indices=[1]))
    tf.summary.scalar(name='MSE', tensor=loss)

### Encapsulating Training Step

In [7]:
with tf.name_scope('Train'):
    train_step = tf.train.GradientDescentOptimizer(learning_rate=.1).minimize(loss=loss)

### Creating File Writer

After initiating a session, all the summaries created should merge togehter and then a file writer should be created

In [8]:
init = tf.global_variables_initializer()
sess = tf.Session()

In [9]:
merged = tf.summary.merge_all()
writer = tf.summary.FileWriter(logdir='logs/', graph=sess.graph)
sess.run(init)

### Iterating and Training

During the training, all the summaries should run through the network and their result should be added using File Writer.

**Colaboratory Note:**
Do not run previous steps multiple times. This breaks the session. If needed, **restart the run time** from `Runtime > Restart runtime...` .

In [10]:
for i in range(10000):
  sess.run(train_step, feed_dict={xp: xs, yp: ys})
  result = sess.run(merged, feed_dict={xp: np.array(xs), yp: np.array(ys)})
  writer.add_summary(result, i)

## Download Data From Colab

Now, the data generating throught this run will be saved in `logs/` directory.

To list the available directories use the following command.

In [11]:
!ls

datalab  logs


you also can archive and compress the result and download them to your local machine.

In [12]:
!tar czvf logs.tar.gz logs

logs/
logs/events.out.tfevents.1530204542.728d78802787


In [13]:
from google.colab import files
files.download('logs.tar.gz')

And remove the log files afterwards.

In [14]:
!rm -rvf logs*

removed 'logs/events.out.tfevents.1530204542.728d78802787'
removed directory 'logs'
removed 'logs.tar.gz'
